# Reccommendation_system

The Colab is the implementation of content-based filtering and collaborative filtering of the cigarette recommendation system from cigarette rating score.
Reference Data : https://www.kaggle.com/mikhailverghese/cigarette-reviews-by-smokers

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Libralies

In [36]:
import pandas as pd
import numpy as np
from sklearn.decomposition import NMF

# Import data and review

In [101]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Dataset/Ciggarat_rec/smokerdata.csv')
data.head(10)

,User,Brand,Variety,Type,Date,Strength,Taste,Price,Rating
0,ghanta,Marlboro,Reds,Regular,2007-06-21,Medium,Tolerable,Fair,3
1,Karawasa,Marlboro,Reds,Regular,2007-06-23,Very Strong,Tolerable,Fair,4
2,Tintedace,Camel,Turkish Gold,Regular,2007-06-23,Medium,Very Poor,High,1
3,Karawasa,Camel,Turkish Gold,Regular,2007-06-23,Strong,Pleasant,Fair,4
4,Tintedace,Newport,Full Flavor,Regular,2007-06-23,Medium,Very Pleasant,Fair,5
5,Karawasa,Marlboro,Blend No. 27,Regular,2007-06-23,Medium,Pleasant,Fair,4
6,Grimey,Camel,No. 9,Regular,2007-06-23,Strong,Poor,Fair,2
7,Grimey,Marlboro,Reds,Regular,2007-06-23,Very Strong,Poor,Fair,3
8,Karawasa,Parliament,Lights (White),Regular,2007-06-23,Medium,Tolerable,Fair,3
9,Grimey,Marlboro,Blend No. 27,Regular,2007-06-23,Weak,Poor,Fair,2


In [38]:
# Identify number of user
User = data['User'].drop_duplicates()
print("The number of reviewers is "+str(len(User)))

The number of reviewers is 2953


In [39]:
# Number of brand 
brand = len(data['Brand'].drop_duplicates())
print("The number of brands is "+str(brand))

The number of brands is 24


จากตารางจะเห็นว่าในแต่ละยี่ห้อจะมีหลายรุ่นย่อย ดังนั้นจะวิเคราะห์จำนวนสินค้าทั้งหมดต้องคำนึงถึงรุ่นย่อยในแต่ละยี่ห้อด้วย

In [40]:
# Joint Brand and Variety together

data['Products'] = data[['Brand', 'Variety']].apply(lambda x: ' '.join(x), axis=1)
data.head()

,User,Brand,Variety,Type,Date,Strength,Taste,Price,Rating,Products
0,ghanta,Marlboro,Reds,Regular,2007-06-21,Medium,Tolerable,Fair,3,Marlboro Reds
1,Karawasa,Marlboro,Reds,Regular,2007-06-23,Very Strong,Tolerable,Fair,4,Marlboro Reds
2,Tintedace,Camel,Turkish Gold,Regular,2007-06-23,Medium,Very Poor,High,1,Camel Turkish Gold
3,Karawasa,Camel,Turkish Gold,Regular,2007-06-23,Strong,Pleasant,Fair,4,Camel Turkish Gold
4,Tintedace,Newport,Full Flavor,Regular,2007-06-23,Medium,Very Pleasant,Fair,5,Newport Full Flavor


In [41]:
# Number of different products  
Products = len(data['Products'].drop_duplicates())
print("The number of products is "+str(Products))

The number of products is 92


# Content base filtering

ต้องทำการหาว่า product แต่ละตัวมีคุณลักษณะอย่างไร โดยดูจากการรีวิวของลูกค้าเป็นหลัก

In [42]:
# Show review of Camel Turkish Gold
data[data['Products']=='Camel Turkish Gold'].head(5)

,User,Brand,Variety,Type,Date,Strength,Taste,Price,Rating,Products
2,Tintedace,Camel,Turkish Gold,Regular,2007-06-23,Medium,Very Poor,High,1,Camel Turkish Gold
3,Karawasa,Camel,Turkish Gold,Regular,2007-06-23,Strong,Pleasant,Fair,4,Camel Turkish Gold
96,thomw,Camel,Turkish Gold,Regular,2007-07-12,Weak,Tolerable,Fair,2,Camel Turkish Gold
178,17yearsold,Camel,Turkish Gold,Regular,2007-08-19,Medium,Very Pleasant,High,5,Camel Turkish Gold
217,reverb10,Camel,Turkish Gold,Regular,2007-08-30,Medium,Tolerable,Fair,3,Camel Turkish Gold


จะเห็นว่าค่าความแรง (Strength) รส (Taste) และราคา (Price) จะแตกต่างกันไปในแต่ละผู้รีวิว ดังนั้นเราจึงต้องทำการหาค่า mode เพื่อเป็นตัวแทนค่าของ product ชนิดนั้นๆ

In [43]:
data_ctg = data[data['Products']=='Camel Turkish Gold']
strength = data_ctg['Strength'].mode()
Taste = data_ctg['Taste'].mode()
Price = data_ctg['Price'].mode()
print("Camel Turkish Gold characteristics")
print("strength : "+str(strength))
print("Taste : "+str(Taste))
print("Price : "+str(Price))

Camel Turkish Gold characteristics
strength : 0    Medium
dtype: object
Taste : 0    Pleasant
dtype: object
Price : 0    Fair
dtype: object


In [44]:
# Find the characteristic of all product

data_brand = data[['Products','Strength','Taste','Price']].copy()
data_brand_dup = data_brand['Products'].drop_duplicates().tolist()
list_brand = []

for i in range(len(data_brand_dup)):
  list_brand_ = (data_brand[data_brand['Products']==data_brand_dup[i]]['Strength'].mode()[0],
  data_brand[data_brand['Products']==data_brand_dup[i]]['Taste'].mode()[0], #mode คือ คนชอบเยอะที่สุด
  data_brand[data_brand['Products']==data_brand_dup[i]]['Price'].mode()[0])
  list_brand.append(list_brand_)

Products_clearn = pd.DataFrame(list_brand,columns=['Strength','Taste','Price'])
Products_clearn['Brand'] = data_brand_dup
Products_clearn.head()

,Strength,Taste,Price,Brand
0,Strong,Pleasant,Fair,Marlboro Reds
1,Medium,Pleasant,Fair,Camel Turkish Gold
2,Strong,Very Pleasant,High,Newport Full Flavor
3,Medium,Very Pleasant,Fair,Marlboro Blend No. 27
4,Weak,Pleasant,Fair,Camel No. 9


สุ่ม User มา 1 คน

In [45]:
user_data = data.iloc[11]
print(user_data)

User                         Lothar
Brand               American Spirit
Variety                     Mediums
Type                        Regular
Date                     2007-06-23
Strength                     Strong
Taste                 Very Pleasant
Price                          High
Rating                            5
Products    American Spirit Mediums
Name: 11, dtype: object


จากข้อมูลพบว่าคุณ Lothar ได้ให้ Rating บุหรี่ American Spirit Mediums ไว้ที่ 5/5 ซึ่งสามารถอนุมานได้ว่าคุณ Lothar ชอบบุหรี่ที่มีคุณลักษณะแบบเดียวกับ American Spirit Mediums ซึ่งมีคุณสมบัติดังนี้

In [46]:
Products_clearn[Products_clearn['Brand']=='American Spirit Mediums']

,Strength,Taste,Price,Brand
7,Medium,Pleasant,High,American Spirit Mediums


ทำการแสดงบุหรี่ที่มีคุณลักษณะเดียวกัน เพื่อเป็นตัวเลือกในการแนะนำให้คุณ Lothar

In [47]:
def content_data(x,y,z):
  Recommend = Products_clearn[(Products_clearn['Strength']==x)&(Products_clearn['Taste']==y)&(Products_clearn['Price']==z)]
  return Recommend

In [48]:
content_data('Medium','Pleasant','High')

,Strength,Taste,Price,Brand
5,Medium,Pleasant,High,Parliament Lights (White)
7,Medium,Pleasant,High,American Spirit Mediums
25,Medium,Pleasant,High,American Spirit Menthol Lights
49,Medium,Pleasant,High,Djarum Specials


#Collaborative filtering

In [49]:
data_new = data[['User','Products','Rating','Strength','Taste','Price']].copy()
data_new.head()

,User,Products,Rating,Strength,Taste,Price
0,ghanta,Marlboro Reds,3,Medium,Tolerable,Fair
1,Karawasa,Marlboro Reds,4,Very Strong,Tolerable,Fair
2,Tintedace,Camel Turkish Gold,1,Medium,Very Poor,High
3,Karawasa,Camel Turkish Gold,4,Strong,Pleasant,Fair
4,Tintedace,Newport Full Flavor,5,Medium,Very Pleasant,Fair


In [50]:
Cmatrix = pd.crosstab(data_new['User'],data_new['Products'],data_new['Rating'],aggfunc='mean')
Cmatrix

Products,305's Full Flavor,American Spirit Full Flavor,American Spirit Lights,American Spirit Mediums,American Spirit Menthol,American Spirit Menthol Lights,American Spirit Organic,American Spirit Perique,American Spirit Ultra Lights,Basic Full Flavor,Benson and Hedges Full Flavor,Benson and Hedges Gold,Benson and Hedges Menthol,Camel Crush,Camel Crush (Bold),Camel Filters Full Flavor,Camel Full Flavor,Camel Lights,Camel Menthol (Cool Burst Green),Camel Menthol (Silver),Camel No. 9,Camel No. 9 Menthol,Camel Reds,Camel Signature Blend Frost,Camel Signature Blend Infused,Camel Turkish Gold,Camel Turkish Royal,Camel Turkish Silver,Camel Wides Filters,Djarum Black,Djarum Specials,Dunhill Full Flavor (Red),KOOL Menthol,KOOL Milds,KOOL XL Green,L&M Full Flavor,L&M Lights,L&M Menthol,Lucky Strike Classic,Lucky Strike Filters,...,Marlboro Menthol Lights,Marlboro Menthol Milds,Marlboro Reds,Marlboro Skyline Menthol,Marlboro Smooth Menthol,Marlboro Southern Cut,Marlboro Special Blend (Black),Marlboro Special Blend (Red),Marlboro Special Blend Gold Lights,Marlboro Special Blend Red,Marlboro Ultra Lights,Marlboro Virginia Blend,Maverick Full Flavor,Maverick Menthol (Silver),Misty Menthol,Nat Sherman Classic,Nat Sherman Hint of Mint,Newport Full Flavor,Newport Menthol,Newport Non-Menthol,Pall Mall Black Menthol,Pall Mall Full Flavor,Pall Mall Lights (Blue),Pall Mall Menthol,Pall Mall Menthol Lights,Pall Mall Reds,Pall Mall Straights,Pall Mall Ultra Lights (Orange),Parliament Full Flavor,Parliament Lights (White),Pyramid Full Flavor,Salem Green Label Full Flavor,Sonoma Full Flavor,USA Gold Full Flavor,Virginia Slims Full Flavor,Virginia Slims Luxury Lights,Virginia Slims Menthol,Winston Full Flavor,Winston Lights,marlboro special blend
User,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
SavvLovesCamel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
!Eureka,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
#1gramma,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
$ixtiesFreak,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
00000,NaN,NaN,NaN,NaN,1.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zoey05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [51]:
Cmatrix = Cmatrix.fillna(0)
num_name_user = list(range(len(User)))
Cmatrix.index = num_name_user
Cmatrix

Products,305's Full Flavor,American Spirit Full Flavor,American Spirit Lights,American Spirit Mediums,American Spirit Menthol,American Spirit Menthol Lights,American Spirit Organic,American Spirit Perique,American Spirit Ultra Lights,Basic Full Flavor,Benson and Hedges Full Flavor,Benson and Hedges Gold,Benson and Hedges Menthol,Camel Crush,Camel Crush (Bold),Camel Filters Full Flavor,Camel Full Flavor,Camel Lights,Camel Menthol (Cool Burst Green),Camel Menthol (Silver),Camel No. 9,Camel No. 9 Menthol,Camel Reds,Camel Signature Blend Frost,Camel Signature Blend Infused,Camel Turkish Gold,Camel Turkish Royal,Camel Turkish Silver,Camel Wides Filters,Djarum Black,Djarum Specials,Dunhill Full Flavor (Red),KOOL Menthol,KOOL Milds,KOOL XL Green,L&M Full Flavor,L&M Lights,L&M Menthol,Lucky Strike Classic,Lucky Strike Filters,...,Marlboro Menthol Lights,Marlboro Menthol Milds,Marlboro Reds,Marlboro Skyline Menthol,Marlboro Smooth Menthol,Marlboro Southern Cut,Marlboro Special Blend (Black),Marlboro Special Blend (Red),Marlboro Special Blend Gold Lights,Marlboro Special Blend Red,Marlboro Ultra Lights,Marlboro Virginia Blend,Maverick Full Flavor,Maverick Menthol (Silver),Misty Menthol,Nat Sherman Classic,Nat Sherman Hint of Mint,Newport Full Flavor,Newport Menthol,Newport Non-Menthol,Pall Mall Black Menthol,Pall Mall Full Flavor,Pall Mall Lights (Blue),Pall Mall Menthol,Pall Mall Menthol Lights,Pall Mall Reds,Pall Mall Straights,Pall Mall Ultra Lights (Orange),Parliament Full Flavor,Parliament Lights (White),Pyramid Full Flavor,Salem Green Label Full Flavor,Sonoma Full Flavor,USA Gold Full Flavor,Virginia Slims Full Flavor,Virginia Slims Luxury Lights,Virginia Slims Menthol,Winston Full Flavor,Winston Lights,marlboro special blend
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,1.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2948,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0

แบ่ง Matrix ออกเป็น 2 Matrix โดยใช้เทคนิค Non-Negative Matrix Factorization

In [106]:
nmf = NMF(10)
nmf.fit(Cmatrix)

NMF(alpha=0.0, beta_loss='frobenius', init=None, l1_ratio=0.0, max_iter=200,
    n_components=10, random_state=None, shuffle=False, solver='cd', tol=0.0001,
    verbose=0)

In [107]:
H_matrix = pd.DataFrame(np.round(nmf.components_,4), columns=Cmatrix.columns)
H_matrix

Products,305's Full Flavor,American Spirit Full Flavor,American Spirit Lights,American Spirit Mediums,American Spirit Menthol,American Spirit Menthol Lights,American Spirit Organic,American Spirit Perique,American Spirit Ultra Lights,Basic Full Flavor,Benson and Hedges Full Flavor,Benson and Hedges Gold,Benson and Hedges Menthol,Camel Crush,Camel Crush (Bold),Camel Filters Full Flavor,Camel Full Flavor,Camel Lights,Camel Menthol (Cool Burst Green),Camel Menthol (Silver),Camel No. 9,Camel No. 9 Menthol,Camel Reds,Camel Signature Blend Frost,Camel Signature Blend Infused,Camel Turkish Gold,Camel Turkish Royal,Camel Turkish Silver,Camel Wides Filters,Djarum Black,Djarum Specials,Dunhill Full Flavor (Red),KOOL Menthol,KOOL Milds,KOOL XL Green,L&M Full Flavor,L&M Lights,L&M Menthol,Lucky Strike Classic,Lucky Strike Filters,...,Marlboro Menthol Lights,Marlboro Menthol Milds,Marlboro Reds,Marlboro Skyline Menthol,Marlboro Smooth Menthol,Marlboro Southern Cut,Marlboro Special Blend (Black),Marlboro Special Blend (Red),Marlboro Special Blend Gold Lights,Marlboro Special Blend Red,Marlboro Ultra Lights,Marlboro Virginia Blend,Maverick Full Flavor,Maverick Menthol (Silver),Misty Menthol,Nat Sherman Classic,Nat Sherman Hint of Mint,Newport Full Flavor,Newport Menthol,Newport Non-Menthol,Pall Mall Black Menthol,Pall Mall Full Flavor,Pall Mall Lights (Blue),Pall Mall Menthol,Pall Mall Menthol Lights,Pall Mall Reds,Pall Mall Straights,Pall Mall Ultra Lights (Orange),Parliament Full Flavor,Parliament Lights (White),Pyramid Full Flavor,Salem Green Label Full Flavor,Sonoma Full Flavor,USA Gold Full Flavor,Virginia Slims Full Flavor,Virginia Slims Luxury Lights,Virginia Slims Menthol,Winston Full Flavor,Winston Lights,marlboro special blend
0,0.0000,0.7235,0.3676,0.0763,0.3005,0.0000,0.3775,0.2826,0.0031,0.1720,0.9334,0.0225,0.0000,0.0000,0.1158,0.0000,0.0000,1.1138,0.1689,0.5141,0.2760,0.2719,0.0175,0.0876,0.1471,2.5635,4.7358,1.1474,2.5166,1.7362,0.2559,0.1343,0.0000,0.0056,0.0975,0.0000,0.0876,0.0000,0.0000,0.1016,...,0.2509,0.0935,0.0000,0.1648,0.0000,0.2160,0.1563,0.1643,0.2180,0.1685,0.1105,0.9707,0.0000,0.0286,0.0000,0.1686,0.0000,0.000,0.0000,0.1173,0.0000,0.0000,0.1157,0.0000,0.0058,0.1045,0.0005,0.0000,0.7406,0.7736,0.0465,0.0000,0.0282,0.0579,0.0000,0.0000,0.0000,0.1324,0.0000,0.0
1,0.0302,0.0460,0.0070,0.0000,0.0683,0.0300,0.0486,0.0224,0.0160,0.0000,0.0281,0.0000,0.1092,0.0000,0.0778,0.0000,0.0000,0.0223,0.0518,0.0000,0.0804,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0295,0.0000,0.0384,0.1579,0.0667,0.0822,0.0000,0.0000,...,0.0901,0.0230,0.0000,0.0632,0.0000,0.0000,0.0319,0.0292,0.0552,0.0227,0.0000,0.0676,0.0183,0.0224,0.0003,0.0000,0.0000,0.000,8.1605,0.0527,0.0098,0.0000,0.1031,0.0667,0.0000,0.0451,0.0936,0.0000,0.0185,0.0000,0.0000,0.0000,0.0154,0.0370,0.0000,0.0894,0.0884,0.0223,0.0823,0.0
2,0.0156,0.0000,0.0000,0.0410,0.0000,0.0102,0.0492,0.0000,0.0240,0.0547,0.0343,0.0209,0.0000,0.0000,0.0519,0.0000,0.2779,0.0394,0.0001,0.1501,0.0218,0.0000,0.1465,0.1567,0.0601,0.0000,0.0000,0.0353,0.1783,0.0002,0.0166,0.0591,0.1415,0.0396,0.0000,0.0494,0.0624,0.0112,0.0000,0.1213,...,0.0341,0.0000,7.5167,0.0000,0.0000,0.0000,0.0000,0.0000,0.0005,0.0283,0.1040,0.2121,0.0695,0.0021,0.0000,0.0346,0.0096,0.000,0.0000,0.0842,0.0424,0.0000,0.0115,0.0252,0.0014,0.0722,0.1383,0.0000,0.0962,0.0000,0.0000,0.0372,0.0000,0.0611,0.0000,0.0718,0.0032,0.0779,0.0664,0.0
3,0.0000,0.0000,0.1793,0.0000,0.0304,0.0524,0.0000,0.0757,0.0150,0.0282,0.0000,0.0000,0.2970,0.0000,0.0005,0.0000,0.0029,0.0000,0.0000,0.0868,0.0391,0.1246,0.0000,0.0202,0.0594,0.0000,0.0000,0.0627,0.1077,0.0641,0.0043,0.0000,0.5013,0.0417,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0762,0.0401,0.0000,0.0708,0.0000,0.0000,0.0000,0.0000,0.0549,0.0000,0.0703,0.0255,0.0400,0.0054,0.0017,0.0000,0.0263,7.342,0.0000,0.0000,0.0000,0.0000,0.0000,0.1446,0.0000,0.0000,0.0934,0.0339,0.0245,0.0316,0.0000,0.1544,0.0000,0.0000,0.0286,0.2831,0.0366,0.1083,0.0824,0.0


In [108]:
W_matrix = pd.DataFrame(np.round(nmf.transform(Cmatrix ),4), columns=H_matrix .index)
W_matrix 

,0,1,2,3,4,5,6,7,8,9
0,0.0000,0.0000,0.0204,0.0000,0.0000,0.1638,0.0000,0.0000,0.0000,0.0790
1,0.0387,0.0058,0.0000,0.0000,0.0197,0.0000,0.0000,0.0294,0.2856,0.0004
2,0.0000,0.0022,0.0012,0.0000,0.0000,0.0001,0.0057,0.0061,0.0000,0.0042
3,0.0024,0.0024,0.0050,0.0000,0.0000,0.0000,0.0315,0.0142,0.0365,0.0347
4,0.0838,0.0011,0.0015,0.5450,0.0192,0.0103,0.0315,0.1101,0.0000,0.0155
...,...,...,...,...,...,...,...,...,...,...
2948,0.0000,0.0000,0.0000,0.0000,0.0077,0.0000,0.1360,0.0000,0.0000,0.0000
2949,0.0000,0.0000,0.0000,0.0000,0.0077,0.0000,0.1360,0.0000,0.0000,0.0000
2950,0.0096,0.0000,0.0000,0.0000,0.0075,0.0000,0.0104,0.0013,0.0000,0.0535
2951,0.0055,0.0000,0.0000,0.0075,0.0000,0.0189,0.0058,0.0024,0.3344,0.0000


คำนวณ Score ของบุหรี่แต่ละชนิดต่อ User แต่ละคน

In [137]:
Cmatrix_new = pd.DataFrame(np.round(np.dot(W_matrix,H_matrix),4), columns=Cmatrix.columns)
Cmatrix_new.index = Cmatrix.index
Cmatrix_new.head(10)

Products,305's Full Flavor,American Spirit Full Flavor,American Spirit Lights,American Spirit Mediums,American Spirit Menthol,American Spirit Menthol Lights,American Spirit Organic,American Spirit Perique,American Spirit Ultra Lights,Basic Full Flavor,Benson and Hedges Full Flavor,Benson and Hedges Gold,Benson and Hedges Menthol,Camel Crush,Camel Crush (Bold),Camel Filters Full Flavor,Camel Full Flavor,Camel Lights,Camel Menthol (Cool Burst Green),Camel Menthol (Silver),Camel No. 9,Camel No. 9 Menthol,Camel Reds,Camel Signature Blend Frost,Camel Signature Blend Infused,Camel Turkish Gold,Camel Turkish Royal,Camel Turkish Silver,Camel Wides Filters,Djarum Black,Djarum Specials,Dunhill Full Flavor (Red),KOOL Menthol,KOOL Milds,KOOL XL Green,L&M Full Flavor,L&M Lights,L&M Menthol,Lucky Strike Classic,Lucky Strike Filters,...,Marlboro Menthol Lights,Marlboro Menthol Milds,Marlboro Reds,Marlboro Skyline Menthol,Marlboro Smooth Menthol,Marlboro Southern Cut,Marlboro Special Blend (Black),Marlboro Special Blend (Red),Marlboro Special Blend Gold Lights,Marlboro Special Blend Red,Marlboro Ultra Lights,Marlboro Virginia Blend,Maverick Full Flavor,Maverick Menthol (Silver),Misty Menthol,Nat Sherman Classic,Nat Sherman Hint of Mint,Newport Full Flavor,Newport Menthol,Newport Non-Menthol,Pall Mall Black Menthol,Pall Mall Full Flavor,Pall Mall Lights (Blue),Pall Mall Menthol,Pall Mall Menthol Lights,Pall Mall Reds,Pall Mall Straights,Pall Mall Ultra Lights (Orange),Parliament Full Flavor,Parliament Lights (White),Pyramid Full Flavor,Salem Green Label Full Flavor,Sonoma Full Flavor,USA Gold Full Flavor,Virginia Slims Full Flavor,Virginia Slims Luxury Lights,Virginia Slims Menthol,Winston Full Flavor,Winston Lights,marlboro special blend
0,0.0041,0.4377,0.0471,0.0181,0.0451,0.0114,0.0744,0.3115,0.0128,0.0445,0.0291,0.0308,0.0214,0.0022,0.0023,0.4291,0.2984,0.0534,0.0083,0.0101,0.0257,0.0251,0.0315,0.0244,0.0120,0.0396,0.0000,0.0127,0.0593,0.0133,0.0114,0.0704,0.0378,0.0255,0.0149,0.1147,0.0038,0.0008,0.8486,0.1114,...,0.0323,0.0053,0.1533,0.0082,0.0000,0.0432,0.0034,0.0000,0.0001,0.0044,0.0146,0.0694,0.0521,0.0009,0.0000,0.1409,0.0108,0.0000,0.0000,0.0098,0.0010,0.0000,0.0047,0.0262,0.0000,0.0224,0.2101,0.0033,0.0558,0.0564,0.0419,0.0392,0.0183,0.0382,0.0000,0.0015,0.0001,0.1898,0.0362,0.0
1,0.0031,0.0283,0.1131,0.0478,0.0239,0.0099,0.0241,0.0286,0.0415,0.0242,0.0679,0.0490,0.0232,0.1131,0.0100,0.0022,0.0005,0.2867,0.0439,0.0320,0.0326,0.0455,0.0026,0.0132,0.0175,0.0992,0.1840,0.0869,0.1539,0.1071,0.0122,0.0053,0.1955,0.0609,0.0181,0.1002,0.1092,0.0005,0.0003,0.0644,...,0.2544,0.1652,0.0000,0.0648,0.1402,0.0117,0.0780,0.0065,0.0268,0.0457,0.0375,0.0668,0.0596,0.0298,0.0000,0.0065,0.0073,0.0000,0.0473,0.0407,0.0004,0.1304,0.4379,0.1902,0.0004,0.0841,0.0026,0.0107,0.1225,0.3008,0.0060,0.0831,0.0307,0.0087,0.0052,0.0219,0.0176,0.0604,0.1580,0.0
2,0.0008,0.0010,0.0019,0.0008,0.0034,0.0030,0.0009,0.0055,0.0011,0.0015,0.0016,0.0002,0.0023,0.0236,0.0018,0.0228,0.0031,0.0034,0.0019,0.0031,0.0036,0.0045,0.0007,0.0014,0.0011,0.0001,0.0000,0.0000,0.0032,0.0083,0.0008,0.0008,0.0082,0.0007,0.0012,0.0089,0.0025,0.0002,0.0005,0.0026,...,0.0021,0.0018,0.0090,0.0031,0.0291,0.0030,0.0023,0.0001,0.0009,0.0011,0.0010,0.0015,0.0010,0.0007,0.0000,0.0001,0.0015,0.0000,0.0180,0.0012,0.0002,0.0000,0.0005,0.0023,0.0000,0.0038,0.0008,0.0007,0.0034,0.0005,0.0003,0.0008,0.0006,0.0018,0.0000,0.0017,0.0002,0.0053,0.0028,0.0
3,0.0038,0.0055,0.0189,0.0072,0.0119,0.0103,0.0037,0.0198,0.0097,0.0076,0.0149,0.0071,0.0073,0.0556,0.0060,0.1885,0.0231,0.0434,0.0104,0.0121,0.0137,0.0154,0.0049,0.0042,0.0050,0.0071,0.0114,0.0084,0.0385,0.0234,0.0036,0.0066,0.0388,0.0096,0.0062,0.0757,0.0198,0.0004,0.0000,0.0218,...,0.0371,0.0238,0.0376,0.0160,0.0677,0.0214,0.0178,0.0005,0.0052,0.0092,0.0086,0.0088,0.0113,0.0050,0.0001,0.0006,0.0036,0.0000,0.0196,0.0084,0.0008,0.0000,0.0574,0.0353,0.0001,0.0304,0.0033,0.0032,0.0284,0.0400,0.0022,0.0162,0.0047,0.0078,0.0006,0.0112,0.0024,0.0465,0.0395,

สร้างระบบแนะนำ

In [135]:
def recomendation(uid,topk):
  res = Cmatrix_new.T[uid].sort_values(ascending=False)[0:topk]
  res = list(res[res>2.5].index) #กำหนดให้แนะนำเฉพาะบุหรี่ที่ rating > 2.5/5
  res = data_new[data_new['Products'].isin(res)]
  res = res.drop_duplicates(subset='Products')
  res = res[:topk]
  res = res[['Products','Strength','Taste','Price']]
  return res

แนะนำสินค้าบุหรี่ให้ user คนที่ 6 โดยกำหนดให้โชว์ 5 อันดับที่แนะนำ แต่เนื่องจากมีเพียง 2 ชนิดที่เกินเงื่อนไข จึงแสดงแค่ 2 ชนิด

In [136]:
user = 6
rec_product = recomendation(user,topk=5)
rec_product

,Products,Strength,Taste,Price
53,Marlboro Smooth Menthol,Medium,Tolerable,High
678,Camel Crush,Medium,Pleasant,Very High


# Conclusion

Colab นี่เป็นการสร้าง Recommendation แบบ Content base filtering ซึ่งใช้ข้อมูลของตัวสินค้าเป็นหลัก และ Collaborative filtering ซึ่งใช้ข้อมูลของลูกค้าที่ซื้อสินค้าเป็นหลักในการสร้างระบบ Recommendation 

* Content base filtering ใช้คุณลักษณะของสินค้าจากการรีวิวของลูกค้าคนอื่นๆ จากนั้นแนะนำสินค้าให้ลูกค้าคนอื่นๆที่มีความชอบในลักษณะนั้นๆของสินค้า
* Collaborative filtering ใช้หลักการ Matrix factorization ผ่าน Library Non-Negative Matrix Factorization (NMF) ของ Scikit learn